In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

from desc import set_device
set_device("gpu")

In [2]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go

In [3]:
from extra_coil_objectives import CoilBounds, SurfaceMatch

In [4]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *
from desc.io import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *
from desc.particles import *
from diffrax import *

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic
from desc.compute.data_index import register_compute_fun
from desc.optimize.utils import solve_triangular_regularized

print_backend_info()

DESC version=0.16.0+36.gb29e1911d.
Using JAX backend: jax version=0.6.2, jaxlib version=0.6.2, dtype=float64.
Using device: NVIDIA GeForce RTX 4080 Laptop GPU (id=0), with 9.43 GB available memory.


# About this Notebook

We will optimize an umbilic coil for HBT hybrid design.

In [5]:
eq = load("eq_final2.h5")
base_coils = load("hbt_default_coils.h5")    # this is a MixedCoilSet

In [6]:

minor_radius = eq.compute("a")["a"]
offset = 1.8 * minor_radius
num_coils = 1  # coils per half field period per coilset
zeta = np.linspace(0, 2 * np.pi, 41)
grid = LinearGrid(rho=[0.0], zeta=zeta, NFP=eq.NFP)
data = eq.axis.compute(["x", "x_s"], grid=grid, basis="xyz")
helical_offset = 0
R0 = 1.0
R = R0 + offset * np.cos(zeta - helical_offset)
Z = offset * np.sin(zeta - helical_offset)

data = jnp.vstack([R, zeta, Z]).T
umbilic_coil = FourierRZCoil.from_values(
    current=-2.1e3,
    coords=data,
    N=10,
    basis="rpz",
)
coil_grid = LinearGrid(N=50)

In [7]:
fig = plot_coils(umbilic_coil)
plot_3d(eq, "|B|", fig=fig)

## Combined Field

In [8]:
field = base_coils.copy()
field.append(umbilic_coil)

## Constraints

In [9]:
params_to_fix_field = [
    # tf coilset
    {"current": True, "r_n": True, "center": True, "normal": True},
    # vf inner coilset
    {"current": True, "r_n": True, "center": True, "normal": True},
    # vf outer coilset
    {"current": True, "r_n": True, "center": True, "normal": True},
    # oh inner coilset
    {"current": True, "r_n": True, "center": True, "normal": True},
    # oh outer coilset
    {"current": True, "r_n": True, "center": True, "normal": True},
    # umbilic coil
    {"current": False, "R_n": False, "Z_n": False, "rotmat": False, "shift": False},
]

In [10]:
k = 7

modes_R = np.vstack(
    (
        [0, 0, 0],
        eq.surface.R_basis.modes[np.max(np.abs(eq.surface.R_basis.modes), 1) > k, :],
    )
)
modes_Z = eq.surface.Z_basis.modes[np.max(np.abs(eq.surface.Z_basis.modes), 1) > k, :]
bdry_constraints = (
    FixBoundaryR(eq=eq, modes=modes_R),
    FixBoundaryZ(eq=eq, modes=modes_Z),
)

constraints = (
    ForceBalance(eq=eq),
    FixPsi(eq=eq),
    FixPressure(eq=eq),
    FixCurrent(eq=eq),
    # FixIota(eq=eq),
    FixParameters(
        field,
        params_to_fix_field,
    ),
)
for con in constraints:
    con.build()

Precomputing transforms


## Objectives



In [11]:
len1 = 2 * np.pi * 1.45
len2 = 2 * np.pi * 1.05

dist1 = np.inf
dist2 = 0.1

objective = ObjectiveFunction(
    (
        BoundaryError(
            eq,
            field,
            deriv_mode="fwd",
            weight=60,
            bs_chunk_size=10,
            B_plasma_chunk_size=10,
        ),
        # CoilSetMinDistance(
        #     field,
        #     bounds=(0.1, 100),
        #     grid=LinearGrid(N=100),
        #     weight=20,
        #     deriv_mode="fwd",
        # ),
        CoilLength(
            field,
            bounds=(0, len2),
            normalize_target=True,
            weight=[0, 0, 0, 0, 0, 5],
            grid=coil_grid,
            deriv_mode="fwd",
        ),
        PlasmaCoilSetMinDistance(
            eq,
            field,
            bounds=(0.04, np.inf),
            normalize_target=True,
            coil_grid=coil_grid,
            eq_fixed=True,
            weight=500,
        ),
        CoilBounds(
            field,
            bounds=(0.0, 0.01),
            normalize_target=True,
            grid=coil_grid,
            weight=[0, 0, 0, 0, 0, 10],
        ),
    ),
    deriv_mode="batched",
)

optimizer = Optimizer("proximal-lsq-exact")

## Optimize

In [12]:
[eq, field_opt], out = optimizer.optimize(
    [eq, field],
    objective,
    constraints + bdry_constraints,
    verbose=3,
    options={},
    copy=True,
    ftol=1e-10,
    xtol=1e-10,
    gtol=1e-10,
    maxiter=0,
)

Building objective: Boundary error
Precomputing transforms
Timer: Precomputing transforms = 156 ms
Building objective: coil length
Precomputing transforms
Timer: Precomputing transforms = 309 ms
Building objective: plasma-coil minimum distance
Building objective: coil bound
Precomputing transforms
Timer: Precomputing transforms = 8.08 ms
Timer: Objective build = 6.56 sec
Timer: Objective build = 1.94 ms
Timer: Objective build = 2.16 ms
Timer: Eq Update LinearConstraintProjection build = 6.05 sec
Timer: Proximal projection build = 1.78 min
Building objective: lcfs R
Building objective: lcfs Z
Building objective: fixed shift
Building objective: fixed rotation
Timer: Objective build = 1.14 sec
Timer: LinearConstraintProjection build = 10.3 sec
Number of parameters: 267
Number of objectives: 6547
Timer: Initializing the optimization = 1.97 min

Starting optimization
Using method: proximal-lsq-exact
Solver options:
------------------------------------------------------------
Maximum Functio